# Predicting Credit Card Approvals

In [1]:
import pandas as pd
import numpy as np
cc_apps=pd.read_csv('D:/DS projects/crx.data',header=None)
cc_apps.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,00202,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,00043,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,00280,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,00100,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,+


In [2]:
cc_apps.describe()
cc_apps.info()
cc_apps.tail()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 16 columns):
0     690 non-null object
1     690 non-null object
2     690 non-null float64
3     690 non-null object
4     690 non-null object
5     690 non-null object
6     690 non-null object
7     690 non-null float64
8     690 non-null object
9     690 non-null object
10    690 non-null int64
11    690 non-null object
12    690 non-null object
13    690 non-null object
14    690 non-null int64
15    690 non-null object
dtypes: float64(2), int64(2), object(12)
memory usage: 86.4+ KB


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
685,b,21.08,10.085,y,p,e,h,1.25,f,f,0,f,g,00260,0,-
686,a,22.67,0.750,u,g,c,v,2.00,f,t,2,t,g,00200,394,-
687,a,25.25,13.500,y,p,ff,ff,2.00,f,t,1,t,g,00200,1,-
688,b,17.92,0.205,u,g,aa,v,0.04,f,f,0,f,g,00280,750,-
689,b,35.00,3.375,u,g,c,h,8.29,f,f,0,t,g,00000,0,-


## Handling the missing values

In [3]:
# Replace the '?'s with NaN
cc_apps=cc_apps.replace('?',np.NaN)
cc_apps.isnull().sum()

0     12
1     12
2      0
3      6
4      6
5      9
6      9
7      0
8      0
9      0
10     0
11     0
12     0
13    13
14     0
15     0
dtype: int64

In [4]:
# Impute the missing values with mean imputation
cc_apps.fillna(cc_apps.mean(),inplace=True)

cc_apps.isnull().sum()



0     12
1     12
2      0
3      6
4      6
5      9
6      9
7      0
8      0
9      0
10     0
11     0
12     0
13    13
14     0
15     0
dtype: int64

In [5]:
#Impute missing values with most frequent value in each column
for col in range(len(cc_apps.columns)):
     if cc_apps[col].dtypes=='object':
         most_frequent=cc_apps[col].value_counts().index[0]
         cc_apps[col]=cc_apps[col].fillna(most_frequent)
cc_apps.isnull().sum()

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
dtype: int64

## Preprocessing data

In [6]:
#Convert categorical data into numeric data with Label Encoder
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

In [7]:
for col in range(len(cc_apps.columns)):
     if cc_apps[col].dtypes=='object':
            cc_apps['{}_cat'.format(col)]=le.fit_transform(cc_apps[col])



In [8]:
df_new=cc_apps.select_dtypes(exclude=['object'])

In [9]:
df_new

,2,7,10,14,0_cat,1_cat,3_cat,4_cat,5_cat,6_cat,8_cat,9_cat,11_cat,12_cat,13_cat,15_cat
0,0.000,1.25,1,0,1,156,1,0,12,7,1,1,0,0,68,0
1,4.460,3.04,6,560,0,328,1,0,10,3,1,1,0,0,11,0
2,0.500,1.50,0,824,0,89,1,0,10,3,1,0,0,0,96,0
3,1.540,3.75,5,3,1,125,1,0,12,7,1,1,1,0,31,0
4,5.625,1.71,0,0,1,43,1,0,12,7,1,0,0,2,37,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,10.085,1.25,0,0,1,52,2,2,4,3,0,0,0,0,90,1
686,0.750,2.00,2,394,0,71,1,0,1,7,0,1,1,0,67,1
687,13.500,2.00,1,1,0,97,2,2,5,2,0,1,1,0,67,1
688,0.205,0.04,0,750,1,20,1,0,0,7,0,0,0,0,96,1


In [10]:
#Split dataset into train and test sets
from sklearn.model_selection import train_test_split

In [11]:
df_new= df_new.drop(['11_cat','13_cat'],axis=1)
np_arr=df_new.to_numpy()

In [12]:
np_arr

array([[ 0.   ,  1.25 ,  1.   , ...,  1.   ,  0.   ,  0.   ],
       [ 4.46 ,  3.04 ,  6.   , ...,  1.   ,  0.   ,  0.   ],
       [ 0.5  ,  1.5  ,  0.   , ...,  0.   ,  0.   ,  0.   ],
       ...,
       [13.5  ,  2.   ,  1.   , ...,  1.   ,  0.   ,  1.   ],
       [ 0.205,  0.04 ,  0.   , ...,  0.   ,  0.   ,  1.   ],
       [ 3.375,  8.29 ,  0.   , ...,  0.   ,  0.   ,  1.   ]])

In [13]:
df_new

,2,7,10,14,0_cat,1_cat,3_cat,4_cat,5_cat,6_cat,8_cat,9_cat,12_cat,15_cat
0,0.000,1.25,1,0,1,156,1,0,12,7,1,1,0,0
1,4.460,3.04,6,560,0,328,1,0,10,3,1,1,0,0
2,0.500,1.50,0,824,0,89,1,0,10,3,1,0,0,0
3,1.540,3.75,5,3,1,125,1,0,12,7,1,1,0,0
4,5.625,1.71,0,0,1,43,1,0,12,7,1,0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,10.085,1.25,0,0,1,52,2,2,4,3,0,0,0,1
686,0.750,2.00,2,394,0,71,1,0,1,7,0,1,0,1
687,13.500,2.00,1,1,0,97,2,2,5,2,0,1,0,1
688,0.205,0.04,0,750,1,20,1,0,0,7,0,0,0,1


In [14]:
X,y=np_arr[:,0:-1],np_arr[:,-1]

In [15]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [16]:
#Scale all values to the range of 0-1
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
rescaledX_train = scaler.fit_transform(X_train)
rescaledX_test = scaler.fit_transform(X_test)

## Fit a logistic regression model to the train set


In [17]:
from sklearn.linear_model import LogisticRegression
logreg=LogisticRegression()

In [18]:
logreg.fit(rescaledX_train,y_train)

D:\PYTHON\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [19]:
#Evaluate performance of the predictive model
from sklearn.metrics import confusion_matrix

In [20]:
y_pred=logreg.predict(rescaledX_test)

In [21]:
print("Accuracy of logistic regression classifier: ", logreg.score(rescaledX_test,y_test))

Accuracy of logistic regression classifier:  0.8260869565217391


In [22]:
print(confusion_matrix(y_test,y_pred))

[[60 10]
 [14 54]]


## Hyperparameter tuning

In [23]:
# Grid search and select the best performing model
from sklearn.model_selection import GridSearchCV

In [24]:
tol = [0.01,0.001,0.0001]
max_iter = [100,150,200]
param_grid={'tol':tol,'max_iter':max_iter}

In [25]:
grid_model = GridSearchCV(estimator=logreg, param_grid=param_grid, cv=5)
rescaledX = scaler.fit_transform(X)
grid_model_result = grid_model.fit(rescaledX, y)
best_score, best_params = grid_model.best_score_,grid_model.best_params_


D:\PYTHON\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\PYTHON\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\PYTHON\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\PYTHON\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\PYTHON\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarn

Best: 0.852174 using {'max_iter': 100, 'tol': 0.01}


D:\PYTHON\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\PYTHON\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [26]:
print("Best: %f using %s" % (best_score, best_params))

Best: 0.852174 using {'max_iter': 100, 'tol': 0.01}
